# Data Description

## Variables no utilizadas

### Id (primera columna sin nombre)

Identificación del registro en el dataset.

- Tipo de variable: numérica.
- No se utilizará como variable porque no aporta información sobre el diamante en sí, y facilitaría el sobreentrenamiento de los modelos que utilicemos.

---

## Variables de entrada

### Quilate (carat)

Peso en quilates del diamante. Un quilate equivale a 0,2 gramos. Junto al corte, color y claridad, forman lo que se denomina "4C": el conjunto de características más importantes a la hora de evaluar la calidad de un diamante.

- Tipo de variable: numérica.
- Transformaciones: ninguna.

### Corte (cut)

Etiqueta que indica la calidad del corte del diamante. El corte es la más importante de las "4C" porque es la característica que tiene mayor influencia en el brillo del diamante.

![Diamond Cut](images/diamond_cut.gif)

- Tipo de variable: categórica.
- Etiquetas posibles (orden creciente): Fair, Good, Very Good, Premium, Ideal.
- Transformaciones: 

### Color (color)

Color del diamante. Es la segunda característica más importante de un diamante, debido a que es lo que el ojo nota luego del destello. El grado de color del diamante, en realidad, se refiere a la ausencia de color en el mismo.

![Diamond Color](images/diamond_color.png)

- Tipo de variable: categórica.
- Etiquetas posibles (orden creciente): J, I, H, G, F, E, D.
- Transformaciones: 

### Claridad (clarity)

Etiqueta que indica qué tan visibles son las imperfecciones y/o inclusiones del diamante. Las imperfecciones son pequeñas características de la superficie del diamante, mientras que las inclusiones se refieren a características internas del mismo. 
La claridad es la menos importante de las 4C. Esto se debe a que las inclusiones son microscópicas y, por ende, no afectan la belleza del diamante de una forma visible.

![Diamond Clarity](images/diamond_clarity.jpg)

- Tipo de variable: categórica.
- Etiquetas posibles (orden creciente): I3, I2, I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF, FL. No hay ejemplos en el dataset de claridad tipo I3, I2 y FL.
- Transformaciones: 

### Porcentaje de profundidad (depth)

Porcentaje de profundidad obtenido dividiendo la altura de un diamante, medida desde el culet hasta la tabla, por su diámetro de filetín promedio. Contribuye a la apariencia, el brillo y el fuego del diamante.

![Diamond Parts](images/diamond_parts.gif)

- Tipo de variable: numérica.
- Transformaciones: ninguna.

### Tabla (table)

El ancho de la tabla expresado como un porcentaje del diámetro promedio del diamante. Medida esencial para el rendimiento ante la luz de un diamante

- Tipo de variable: numérica.
- Transformaciones: ninguna.

### Longitud (x)

Longitud en milímetros del diamante.

- Tipo de variable: numérica.
- Transformaciones: ninguna.

### Ancho (y)

Ancho en milímetros del diamante.

- Tipo de variable: numérica.
- Transformaciones: ninguna.

### Profundidad (z)

Profundidad en milímetros del diamante.

- Tipo de variable: numérica.
- Transformaciones: ninguna.

---

## Variable de salida

### Precio (price)

Precio del diamante expresado en dólares (EEUU).

- Tipo de variable: categórica.
- Transformaciones: 
    1. Crear una nueva columna, llamada "price_category", que representa las etiquetas correspondientes al rango de precio al que pertenezcan los precios de los diamantes. **Esta nueva columna será la variable de salida a predecir**.
    2. 
- Etiquetas posibles: Menor a 1000, Entre 1000 y 3000, Entre 3000 y 6000, Entre 6000 y 10000, Mayor a 10000.

# Descripción de los datos

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sbn
plt.rcParams['figure.figsize'] = 16, 6

In [ ]:
# Cargar dataset
dataset = pd.read_csv('data/diamonds.csv', index_col=0)

#### Forma del dataset

In [ ]:
filas, columnas = dataset.shape
print(f'El dataset tiene {filas} filas y {columnas} columnas.')

#### Primeras filas del dataset

In [ ]:
dataset.head()

#### Métricas de las variables numéricas del dataset

In [ ]:
dataset.describe()

#### Cantidad de valores nulos por atributo del dataset

In [ ]:
dataset.isnull().sum().to_frame('Cantidad de valores Nulos')

#### Histograma de precios del dataset

In [ ]:
ax = dataset.price.plot.hist(bins=50, title='Histograma de precios del dataset', grid=True)
ax.title.set_size(20)
ax.set_xlabel('Precio (Dólares)', fontsize=12)
ax.set_ylabel('Frecuencia', fontsize=12);

#### Transformación de la variable de salida Precio

In [ ]:
dataset.price.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).to_frame('Precio')

In [ ]:
price_category_labels = [
    'Menor a 1000', 
    'Entre 1000 y 3000', 
    'Entre 3000 y 6000', 
    'Entre 6000 y 10000', 
    'Mayor a 10000'
]
price_limits = [0, 1000, 3000, 6000, 10000, 20000]

dataset['price_category'] = pd.cut(dataset.price, price_limits, labels=price_category_labels)

# Análisis detallado de variables

### Balanceo de la variable de salida

In [ ]:
ax = dataset.price_category.value_counts().plot.pie(autopct='%1.0f%%', figsize=(10,10))
ax.set_ylabel('')
ax.set_title('Balanceo de etiquetas de Precios', fontsize=20);

Como se observa en el gráfico, en general, la proporción de instancias en cada categoría es similar y no debería afectar a los modelos clasificadores que utilicemos. 

La excepción se encuentra marcada en las etiquetas "Entre 6000 y 10000" y "Mayor a 10000". Se decidió efectuar esta subdivisión ya que, si bien la proporción era adecuada para la división (22%), el rango de precios que comprendía era demasiado amplio y perdía sentido interpretativo.

## Comportamiento de variables de entrada y relación con variable de salida

### Quilates

In [ ]:
_, data = dataset.boxplot(column=['carat'], return_type='both', vert=False, figsize=(16, 4));

In [ ]:
outliers = len(data['fliers'][0].get_xdata())
print(f'Cantidad de valores atípicos: {outliers}')
print(f'Porcentaje de valores atípicos: {outliers / dataset.shape[0] * 100:.2f}%')

In [ ]:
dataset.boxplot(column=['carat'], by='price_category');

In [ ]:
carat_dataframe = pd.DataFrame()
carat_dataframe['price'] = dataset['price']
carat_limits = [0, 0.4, 0.6, 0.8, 1, 2, 3, 4, 5, 6]
carat_dataframe['carat_range'] = pd.cut(dataset.carat, carat_limits)
ax = carat_dataframe.groupby(['carat_range',]).mean()[['price',]].plot.bar(rot=0, legend=False, grid=True)
ax.set_title('Precio promedio por rango de quilates', fontsize=20)
ax.set_xlabel('Rango de Quilates', fontsize=12)
ax.set_ylabel('Precio promedio', fontsize=12);

### Corte

In [ ]:
dataset.cut.value_counts().plot.pie(autopct='%1.0f%%');

In [ ]:
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']

dataset.groupby(['cut', 'price_category',]).size().unstack().reindex(cut_order).T.plot.bar(stacked=True, rot=45);

In [ ]:
dataset.groupby(['cut',]).mean()[['price']].reindex(cut_order).plot.bar();

In [ ]:
dataset.groupby(['cut',]).mean

### Color

In [ ]:
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D', ]
dataset.color.value_counts().reindex(color_order).plot.pie(autopct='%1.0f%%');

In [ ]:
dataset.groupby(['color', 'price_category',]).size().unstack().reindex(color_order).T.plot.bar(stacked=True, rot=45);

### Claridad

In [ ]:
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
dataset.clarity.value_counts().reindex(clarity_order).plot.pie(autopct='%1.0f%%');

In [ ]:
dataset.groupby(['clarity', 'price_category',]).size().unstack().reindex(clarity_order).T.plot.bar(stacked=True, rot=45);

### Porcentaje de profundidad

In [ ]:
_, data_depth = dataset.boxplot(column=['depth'], return_type='both', vert=False, figsize=(16, 4));

In [ ]:
outliers_depth = len(data_depth['fliers'][0].get_xdata())
print(f'Cantidad de valores atípicos: {outliers_depth}')
print(f'Porcentaje de valores atípicos: {outliers_depth / dataset.shape[0] * 100:.2f}%')

In [ ]:
dataset.boxplot(column=['depth'], by='price_category');